# Character-level LSTM with PyTorch

An RNN model will be trained to generate new text character by character.

In [1]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Training data

In [3]:
with open('html_physics_book.txt', 'r') as f:
    text = f.read()

### Characters tokenization

We need to be able to convert each character into an integer token.

In [4]:
chars_in_text = tuple(set(text))

int2char = dict(enumerate(chars_in_text))
char2int = {ch: ii for ii, ch in int2char.items()}

encoded_text = np.array([char2int[ch] for ch in text])

In [5]:
# Encoding and then decoding produces the initial character
print(int2char[char2int['X']])

X


The LSTM accepts only one-hot encoded vectors, so we prepare a function that takes as input an array of encoded characters and outputs an array of one-hot vectors.

In [6]:
def to_one_hot(char_encoding):
    one_hot = np.zeros(len(int2char))
    one_hot[char_encoding] = 1
    return one_hot

# Test
to_one_hot(12)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Making training mini-batches

We want to create both the input and target arrays, with the targets being the same as the inputs, but shifted over one character.

In [7]:
# Divide the `encodings_arr` array into batches.
def get_batches(encodings_arr, sequences_per_batch, chars_per_sequence):
    batch_size = sequences_per_batch * chars_per_sequence
    number_of_batches = len(encodings_arr)//batch_size
    
    # Keep only enough characters to make full batches
    encodings_arr = encodings_arr[:number_of_batches * batch_size]
    
    # Split the array into sequences by reshaping it into `sequences_per_batch` rows.
    encodings_arr = encodings_arr.reshape((sequences_per_batch, -1))
    
    for n in range(0, encodings_arr.shape[1], chars_per_sequence):        
        # The features
        x = encodings_arr[:, n:n+chars_per_sequence]
        
        # The targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], encodings_arr[:, n+chars_per_sequence]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], encodings_arr[:, 0]
        
        yield x, y

#### Test the mini-batches generation

In [8]:
first_batch = next(get_batches(encoded_text, 3, 100))
print(f'Every mini-batch is a tuple of {len(first_batch)} arrays.')
print(f'First one contains features and in this case has shape {first_batch[0].shape}')
print(f'Second one contains targets and in this case has shape {first_batch[1].shape}')

Every mini-batch is a tuple of 2 arrays.
First one contains features and in this case has shape (3, 100)
Second one contains targets and in this case has shape (3, 100)


In [9]:
x, y = first_batch
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[91 72 12 47 24 56 79 48 61 14]
 [ 8 13 39 14 65  8 13 39 46 14]
 [ 6 13 11 76 38 39 14  5 76 14]]

y
 [[72 12 47 24 56 79 48 61 14 13]
 [13 39 14 65  8 13 39 46 14  8]
 [13 11 76 38 39 14  5 76 14  6]]


The data is correctly shifted over one step for `y`.

---
## Model definition

In [10]:
class CharRNN(nn.Module):
    def __init__(self, n_hidden=256, n_layers=2, drop_prob=0.5):
        super().__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden

        char_embedding_size = len(int2char) # size of the one-hot vectors representing characters
        self.lstm = nn.LSTM(char_embedding_size, n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, char_embedding_size)
        
        self.init_weights()
      
    
    def forward(self, x, hc):
        # Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        x = self.dropout(x)
        
        # Stack up LSTM outputs
        x = x.reshape(x.size()[0] * x.size()[1], self.n_hidden)
        
        x = self.fc(x)
        
        # Return x and the hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character and the hidden state, predict the next character.
        
            Returns the predicted character and the new hidden state.
        '''
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[to_one_hot(char2int[char])]], dtype=np.float32)
        
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([h_item.data for h_item in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(int2char))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return int2char[char], h
    
    def init_weights(self):
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())
        

## Network training

In [11]:
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
        net.cuda()
    
    counter = 0
    
    for e in range(epochs):
        
        h = net.init_hidden(n_seqs)
        
        for x, y in get_batches(data, n_seqs, n_steps):
            
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = np.array([
                [to_one_hot(c) for c in seq] for seq in x
            ], dtype=np.float32)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            
            loss = criterion(
                output,
                targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor if cuda else torch.LongTensor)
            )

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    
                    # One-hot encode our data and make them Torch tensors
                    x = np.array([
                        [to_one_hot(c) for c in seq] for seq in x
                    ], dtype=np.float32)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(
                        output,
                        targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor if cuda else torch.LongTensor)
                    )
                
                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [12]:
net = CharRNN(n_hidden=512, n_layers=2)

print(net)

CharRNN(
  (lstm): LSTM(94, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=94, bias=True)
)


In [ ]:
train(
    net,
    encoded_text,
    epochs=50,
    n_seqs=128,
    n_steps=300,
    lr=3e-4,
    cuda=torch.cuda.is_available(),
    print_every=10
)

Epoch: 1/50... Step: 10... Loss: 3.6033... Val Loss: 3.6507
Epoch: 1/50... Step: 20... Loss: 3.4519... Val Loss: 3.4675
Epoch: 2/50... Step: 30... Loss: 3.3936... Val Loss: 3.4186
Epoch: 2/50... Step: 40... Loss: 3.3413... Val Loss: 3.3400
Epoch: 3/50... Step: 50... Loss: 3.2932... Val Loss: 3.2549
Epoch: 3/50... Step: 60... Loss: 3.2546... Val Loss: 3.1592
Epoch: 3/50... Step: 70... Loss: 3.1783... Val Loss: 3.0768
Epoch: 4/50... Step: 80... Loss: 3.1183... Val Loss: 2.9830
Epoch: 4/50... Step: 90... Loss: 3.0429... Val Loss: 2.9180
Epoch: 5/50... Step: 100... Loss: 2.9827... Val Loss: 2.8392
Epoch: 5/50... Step: 110... Loss: 2.8908... Val Loss: 2.7719
Epoch: 5/50... Step: 120... Loss: 2.8150... Val Loss: 2.7001
Epoch: 6/50... Step: 130... Loss: 2.7347... Val Loss: 2.6384
Epoch: 6/50... Step: 140... Loss: 2.6860... Val Loss: 2.6240
Epoch: 7/50... Step: 150... Loss: 2.6202... Val Loss: 2.5507
Epoch: 7/50... Step: 160... Loss: 2.5734... Val Loss: 2.5088
Epoch: 8/50... Step: 170... Loss:

After training, we'll save the model so we can load it again later if we need to.

In [17]:
checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict()}

with open('trained_model.net', 'wb') as f:
    torch.save(checkpoint, f)

## Sampling

To sample from the trained model, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. By keeping doing this we'll generate a bunch of text.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text.

In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [18]:
def generate_sample(net, sample_length, prime='<div>', top_k=None, cuda=False):
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    
    h = net.init_hidden(1)
    
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    for ii in range(sample_length):
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [19]:
sample = generate_sample(net, 4000, top_k=5, cuda=True)

print(sample)

<div>
           </p>
      </i>
      </p>
    <h4>Important Topiss</h4>
    <p>16. At that a storce of the sercaly of a pliseding an the maghet the camper subentins the contens of whine the cirmen supents ander and teren to tome tount to the farment to be the sures of whell whice it cust of the caler it of enely in the pondere the ponction is a colles than a porend calle ot an ilachod ator the simcens of the millo to ater inderenting the plessupe of thromes thas tore ated to cand to a pan at in o serant it in suched to thres, at ar plactivice the fremens of the rengition and of the sumen ones is of the ratsument of what the sictions a calterace for the mare of a surto is caller to ater, the cearse than of the reftenting the conconter conser silling antele work whac it cormatusting the somperition. The collents thas that in to te sorman at recusters of to the perance ater to than a conducin of whice the conture of a conse tho ele the soull to thic the selistric formeng a forment is. <